In [ ]:
!pip install mlflow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade mlflow>=2.3.0

In [ ]:
!pip install pyngrok -q

In [ ]:
def set_mlflow():
  get_ipython().system_raw("mlflow ui --port 5000 &")
  ngrok.kill()
  NGROK_AUTH_TOKEN = "your code"
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
  ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
  print("MLflow Tracking UI:", ngrok_tunnel.public_url)
  return ngrok_tunnel.public_url

In [ ]:
!pip install -q kaggle

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn import preprocessing

import os
from google.colab import drive
from kaggle.api.kaggle_api_extended import KaggleApi
import json
from pyngrok import ngrok, conf
import mlflow
from datetime import datetime, timedelta

## Load Data from Kaggle

In [ ]:
#Read creds
drive.mount('/content/drive')

#Get the creds for Kaggle for the Kaggle api
creds_path = ('/content/drive/MyDrive/Projects/Creds/kaggle.json')

Mounted at /content/drive


In [ ]:
with open(creds_path, 'r') as f:
    creds = json.load(f)

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(creds, f)

In [ ]:
# Set permissions
!chmod 600 ~/.kaggle/kaggle.json

# Verify setup
!kaggle datasets list -s "ieee fraud"

# STEP 2: DOWNLOAD DATASET
# Download the dataset (this is a known mirror of the original competition data)
!kaggle datasets download -d kartik2112/fraud-detection

# Unzip the files (creates a 'fraud-detection' folder)
!unzip fraud-detection.zip -d fraud-detection

ref                                                        title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
mlg-ulb/creditcardfraud                                    Credit Card Fraud Detection                          69155672  2018-03-23 01:17:27.913000         902182      12240  0.85294116       
muhakabartay/yourallmodelsdata                             IEEE-CIS Fraud Detection Models Data                 29768007  2019-09-18 07:57:04.473000            684         17  1.0              
whenamancodes/fraud-detection                              Fraud Detection                                      69155672  2022-09-12 11:54:40.550000          10601        118  1.0              
kyakovlev/ieee-submissions-and

In [ ]:
# List downloaded files
!ls -la fraud-detection/

total 489852
drwxr-xr-x 2 root root      4096 Jun 16 08:07 .
drwxr-xr-x 1 root root      4096 Jun 16 08:07 ..
-rw-r--r-- 1 root root 150354339 Aug  5  2020 fraudTest.csv
-rw-r--r-- 1 root root 351238196 Aug  5  2020 fraudTrain.csv


## Load for training from pandas

In [ ]:
train = pd.read_csv("fraud-detection/fraudTrain.csv")
test = pd.read_csv("fraud-detection/fraudTest.csv")

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [ ]:
def data_prep(data):
  #All the steps to data training
  data.drop(['Unnamed: 0','cc_num','first','last','street','zip','trans_num','lat','long','merchant','merch_lat','merch_long','gender','unix_time','category','city'],inplace=True,axis=1)

  data['trans_date_trans_time']= pd.to_datetime(data['trans_date_trans_time'])
  data['month'] = data['trans_date_trans_time'].dt.month
  data['month'] = data['month'].astype(int)
  data['dob']= pd.to_datetime(data['dob'])
  data['age'] = (data['trans_date_trans_time'] - data['dob']).dt.days/365
  data["job"]= data["job"].astype(str)

  data.drop(['trans_date_trans_time','dob'],inplace=True,axis=1)

  label_encoder = preprocessing.LabelEncoder()
  data["job"]= label_encoder.fit_transform(data["job"])
  data["state"]= label_encoder.fit_transform(data["state"])

  return data

In [ ]:
train = data_prep(train)

In [ ]:
test['is_fraud'].value_counts()

,count
is_fraud,
0,553574
1,2145


In [ ]:
print("Class Distribution:")
print(train['is_fraud'].value_counts())
print(f"Percentage: \n{train['is_fraud'].value_counts(normalize=True) * 100}")

# Imbalance ratio (%)
imbalance_ratio = train['is_fraud'].value_counts().min() / train['is_fraud'].value_counts().max()
print(f"Imbalance ratio: {imbalance_ratio:.3f}")

Class Distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Percentage: 
is_fraud
0    99.421135
1     0.578865
Name: proportion, dtype: float64
Imbalance ratio: 0.006


In [ ]:
# Split instances into majority vs minority class/classes
df_majority = train[train['is_fraud'] == 0]
df_minority = train[train['is_fraud'] == 1]

# Undersampling: we keep as many majority instances (n) as minority ones
df_majority_downsampled = df_majority.sample(n=len(df_minority),random_state=42)
df_balanced = pd.concat([df_majority_downsampled, df_minority])

print(f"Original dataset: {len(train)}")
print(f"Balanced dataset: {len(df_balanced)}")

Original dataset: 1296675
Balanced dataset: 15012


In [ ]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15012 entries, 811778 to 1295733
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   amt       15012 non-null  float64
 1   state     15012 non-null  int64  
 2   city_pop  15012 non-null  int64  
 3   job       15012 non-null  int64  
 4   is_fraud  15012 non-null  int64  
 5   month     15012 non-null  int64  
 6   age       15012 non-null  float64
dtypes: float64(2), int64(5)
memory usage: 938.2 KB


In [ ]:
# Imbalance ratio (%)
imbalance_ratio = df_balanced['is_fraud'].value_counts().min() / df_balanced['is_fraud'].value_counts().max()
print(f"Imbalance ratio: {imbalance_ratio:.3f}")

Imbalance ratio: 1.000


## Model Development

In [ ]:
uri = set_mlflow()
mlflow.set_tracking_uri(uri=uri)

MLflow Tracking UI: https://218d-35-247-53-211.ngrok-free.app


In [ ]:
mlflow.set_experiment("Fraud-detection")

2025/06/16 08:07:55 INFO mlflow.tracking.fluent: Experiment with name 'Fraud-detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/474008413382543543', creation_time=1750061275371, experiment_id='474008413382543543', last_update_time=1750061275371, lifecycle_stage='active', name='Fraud-detection', tags={}>

In [ ]:
df_balanced.head()

,amt,state,city_pop,job,is_fraud,month,age
811778,86.19,4,92043,85,0,12,58.657534
1101718,85.81,24,1019,15,0,4,41.213699
800013,99.30,38,184,444,0,12,74.131507
398945,42.09,26,743,489,0,6,47.186301
207455,4.24,42,87124,486,0,4,32.186301


In [ ]:
!pip install xgboost

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 10.6 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
import sklearn
from sklearn.metrics import accuracy_score,confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, train_test_split
import optuna

In [ ]:
mlflow.xgboost.autolog()

In [ ]:
# Split data into X and y
X = df_balanced.drop(columns=['is_fraud'], axis=1)
y = df_balanced['is_fraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [ ]:
y_train.value_counts(normalize=True)

,proportion
is_fraud,
0,0.5
1,0.5


In [ ]:
dmatrix_train = xgb.DMatrix(data=X_train, label=y_train)
dmatrix_test = xgb.DMatrix(data=X_test,label=y_test)

In [ ]:
def objective(trial, dtrain, dvalid):
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial,dmatrix_train, dmatrix_test ), n_trials=100, timeout=600)

[I 2025-06-16 09:01:02,975] A new study created in memory with name: no-name-5dabe8e7-6184-47cd-9ce1-6628cb2258ed
2025/06/16 09:01:03 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0cfa02f5676c41aa87ff8db084552949', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/06/16 09:01:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as dou

🏃 View run loud-bird-605 at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543/runs/0cfa02f5676c41aa87ff8db084552949
🧪 View experiment at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543


2025/06/16 09:01:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '593de230dc89402cae55febd9f0bc6b9', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/06/16 09:01:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

🏃 View run invincible-hog-221 at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543/runs/593de230dc89402cae55febd9f0bc6b9
🧪 View experiment at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543


2025/06/16 09:01:18 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '06b3adad12164de696ea83119d8ed6ab', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/06/16 09:01:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

...

🏃 View run victorious-fox-73 at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543/runs/034a6a1cd95643ff996dc9fb1c32ce73
🧪 View experiment at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543


2025/06/16 09:10:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7582a505cd7e4ba5a86e4f47674ab046', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/06/16 09:10:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

🏃 View run orderly-dog-711 at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543/runs/7582a505cd7e4ba5a86e4f47674ab046
🧪 View experiment at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543


2025/06/16 09:11:00 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '82e78f4293c8414e85a9bfade2360185', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/06/16 09:11:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

🏃 View run welcoming-hare-167 at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543/runs/82e78f4293c8414e85a9bfade2360185
🧪 View experiment at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543


In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Number of finished trials:  79
Best trial:
  Value: 0.9091918294849023
  Params: 
    booster: dart
    lambda: 4.753471494267503e-06
    alpha: 0.3266027192775913
    subsample: 0.8376880644393908
    colsample_bytree: 0.8677921489329279
    max_depth: 9
    min_child_weight: 5
    eta: 9.787927711840947e-08
    gamma: 0.0001884345722651077
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: forest
    rate_drop: 2.7505515477792895e-05
    skip_drop: 3.901305189725552e-06


In [ ]:
model = xgb.train(params = trial.params, dtrain= dmatrix_train)

2025/06/16 09:54:28 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1a04683482514468b9e63a4cd8d16e13', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/06/16 09:54:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.11/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https:/

🏃 View run delicate-stoat-405 at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543/runs/1a04683482514468b9e63a4cd8d16e13
🧪 View experiment at: https://218d-35-247-53-211.ngrok-free.app/#/experiments/474008413382543543


In [ ]:
test_predictions = model.predict(dmatrix_test)
round_test_predictions = [round(p) for p in test_predictions]
accuracy_score(y_test,round_test_predictions)

0.8960923623445826